# Week 7 - MODIS H4  Data in Python 

This notebook reviews how to open a process MODIS data stored in  
HDF4 format using Python .


In [ ]:
import os
import warnings

# import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import rioxarray as rxr
from rasterio.plot import plotting_extent
from shapely.geometry import mapping, box
import geopandas as gpd
import earthpy as et
import earthpy.plot as ep

warnings.simplefilter('ignore')

# Get the data
et.data.get_data('cold-springs-modis-h4')
# This download is for the fire boundary
et.data.get_data('cold-springs-fire')
os.chdir(os.path.join(et.io.HOME,
                      'earth-analytics',
                      'data'))

# This will make rioxarray run faster
rioxarray_option = rxr.set_options(export_grid_mapping=False)

## Hierarchical Data Formats - HDF4 - EOS in Python

You can use gdal or rasterio (which wraps around gdal) to open hdf4 data. 
In this lesson you will use rasterio. Similar to opening **.tif** files using rasterio, 
you will use a context manager to open hdf4 files. However because the data are 
nested, you will see that loops will become important to open and explore the 
reflectance data stored within the h4 file. 

To begin, create a path to your hdf file.

In [ ]:
# Create a path to the pre-fire MODIS h4 data
modis_pre_path = os.path.join("cold-springs-modis-h5",
                              "07_july_2016",
                              "MOD09GA.A2016189.h09v05.006.2016191073856.hdf")


## Open HDF Files Using Open Source Python - rioxarray

HDF files are hierarchical and self describing (the metadata is contained 
within the data). Because the data are hierarchical, you will have to loop
through the main dataset and the subdatasets nested within the main dataset 
to access the reflectance data (the bands) and the qa layers. 

Below  you  explore  and  then open the h4 file. Notice that the data have 
some metatadata associated with it. 

In [ ]:
modis_pre_path

The command below runs `gdalinfo` which allows you to view the metadata.
Calling a command using  `!` in  notebooks runs a bash command!

NOTE - this will ONLY work if `gdal` is installed and  
recognized properly on your computer.

In [ ]:
# View file metadata using bash
!gdalinfo cold-springs-modis-h5/07_july_2016/MOD09GA.A2016189.h09v05.006.2016191073856.hdf

In [ ]:
# This returns a list of two objects
modis = rxr.open_rasterio(modis_pre_path,
                          masked=True)


modis

By adding two output variables, open_rasterio will save
the reflectance / band data in a separate object from
the qa (quality assurance) data.

In [ ]:
# By adding two output variables,
modis_qa, modis_data = rxr.open_rasterio(modis_pre_path,
                                         masked=True)

The fist object returned contains all of the qa (quality assurance) 
and metadata layers.


In [ ]:
# The first object is a qa object
modis_qa

In [ ]:
# The second contains your spectral data - ie the "bands"
modis_data

##  Use Variables and Groups to Open Subsets of Your Data

To access the spatial information stored within your H4 file, you will need 
to loop through the subdatasets. Below you open a connection to the main h4 file, 
then you loop through each subdataset in the file. The files with this pattern 
in the name:

`sur_refl_b01_1`  

are the bands which contain surface reflectance data. 

* **sur_refl_b01_1:** MODIS Band One
* **sur_refl_b02_1:** MODIS Band Two

Below you loop through and print the name of each subdataset in the file.
Notice that there are some other layers in the file as well including the 
`state_1km` layer which contains the QA (cloud and quality assurance) information.

### Use  the  Group  Parameter  to Grab One Of the Sub Groups

You can use groups to  grab an entire subgroup rather than just 
specific layers or variables.
Notice below you get an array with ALL of the reflectance bands.

In [ ]:
# Notice that here, you get a single xarray object with just the bands that
#  you want to work with
rxr.open_rasterio(modis_pre_path,
                  masked=True,
                  group="MODIS_Grid_500m_2D").squeeze()

### Select By  Variables

In [ ]:
# Open just the bands that you want to process
desired_bands = ["sur_refl_b01_1",
                 "sur_refl_b02_1",
                 "sur_refl_b03_1",
                 "sur_refl_b04_1",
                 "sur_refl_b07_1"]
# Notice that here, you get a single xarray object with just the bands that
# you want to work with
modis_bands = rxr.open_rasterio(modis_pre_path,
                                masked=True,
                                variable=desired_bands).squeeze()
modis_bands

##  Clip The Data

There are several different ways to clip your data. While for this assignment,
I suggest  that  you  use the same approach used in `open_clean_bands()` 
shown last week for class. If you are interested below there  is more  
information about a few different approaches.

In [ ]:
# Open the clip extent
fire_boundary_path = os.path.join("cold-springs-fire",
                                  "vector_layers",
                                  "fire-boundary-geomac",
                                  "co_cold_springs_20160711_2200_dd83.shp")
fire_boundary = gpd.read_file(fire_boundary_path)

Note that the data are in different CRS'

In [ ]:
print("The fire boundary crs is", fire_boundary.crs)
print("The MODIS crs is", modis_bands.rio.crs)

In [ ]:
# %%timeit  #  Uncomment this IF you want to do a speed test
modis_bands_not_clipped = rxr.open_rasterio(modis_pre_path,
                                            masked=True,
                                            variable=desired_bands)

fire_reproj = fire_boundary.to_crs(modis_bands.rio.crs).total_bounds

modis_clip = modis_bands.rio.clip_box(*fire_reproj).squeeze()
modis_clip

####  Speed Tests For Clipping Data

For those of you that want to see we tested two approaches  here and  
did find that repojecting using the `crs=` parameter is much slower  
and more memory intensive than the approach above.

For your homework either approach is ok!

In [ ]:
# %%timeit
# Clip and reproject the fire boundary at the same time
# this  uses more ram and is slower
crop_bound_box = [box(*fire_boundary.total_bounds)]

modis_clip_1 = rxr.open_rasterio(modis_pre_path,
                                 masked=True,
                                 variable=desired_bands).rio.clip(crop_bound_box,
                                                                  crs=fire_boundary.crs,
                                                                  from_disk=True)

In [ ]:
modis_clip

In [ ]:
# Flatten the data (here you lose the band names)
modis_clip.to_array(dim="band")

##  Select Bands to Turn Into Array

For  RGB or CIR  plotting,  you may want to select only the bands
needed for plotting

In [ ]:
# Select bands 1 and 2 and turn into array
rgb_bands = ['sur_refl_b01_1',
             'sur_refl_b03_1',
             'sur_refl_b04_1']
modis_rgb_xr = modis_clip[rgb_bands].to_array()
modis_rgb_xr

### Plot All MODIS Bands with EarthPy

You are now ready to plot your data using earthpy. Notice below that the 
images look washed out and there are large negative values in the data. 

This might be a good time to consider cleaning up your data by addressing 
`nodata` values.

In [ ]:
ep.plot_bands(modis_rgb_xr.values,
              scale=False,
              figsize=(10, 5),
              title=rgb_bands)
plt.show()

### RGB Image of MODIS Data Using EarthPy

Once you have your data cleaned up, you can plot an RGB image of your data 
to ensure that it looks correct!

[This lesson will help you remember what bands to use.](https://www.earthdatascience.org/courses/use-data-open-source-python/multispectral-remote-sensing/intro-multispectral-data/)

In [ ]:
def clean_array_plot(xr_obj):
    # This function takes a single xarray object as an input and produces a
    # cleaned numpy array output for plotting
    # BEGIN SOLUTION
    """
    Take an  xarray object and replace null  values with a mask for plotting

    Parameters
    ----------
    xr_obj : xarray object

    Returns
    -------
    A masked numpy array 

    """
    # END SOLUTION

    return ma.masked_array(xr_obj.values,  xr_obj.isnull())

In [ ]:
modis_rgb_clean = clean_array_plot(modis_rgb_xr)

# Plot MODIS RGB image -Note that this looks weird only because
# the  data  are clipped to such  a "small" extent
ep.plot_rgb(modis_rgb_clean,
            rgb=[0, 1, 2],
            title='RGB Image of MODIS Data',
            figsize=(7, 7))

plt.show()

In [ ]:
modis_rgb_unclipped = modis_bands_not_clipped[rgb_bands].to_array().squeeze()
modis_rgb_unclipped

In [ ]:
# Plot unclipped data - RGB (just a demo)
modis_rgb_clean_not_clipped = clean_array_plot(modis_rgb_unclipped)

# Plot MODIS RGB image -Note that this looks weird only because
# the data are clipped to such a "small" extent
ep.plot_rgb(modis_rgb_clean_not_clipped,
            rgb=[0, 1, 2],
            title='RGB Image of MODIS Data',
            figsize=(7, 7))

plt.show()

##  Metadata for MODIS

If you look at the <a href="{{ site.url }}/courses/earth-analytics-python/multispectral-remote-sensing-modis/modis-remote-sensing-data-in-python/"> table in the MODIS documentation or that is on the earthdatascience.org intermediate textbook,</a> you will see that the
range of value values for MODIS spans from **-100 to 16000**. 

There is also a fill or no data value **-28672** to consider.   

NOTE:

In [ ]:
# View entire metadata object for a MODIS band
modis_clip.rio.crs

In [ ]:
#  View variables in object
modis_clip.rio.vars

In [ ]:
# View data resolution
modis_clip.rio.resolution()

##  Function  To  Open  and Clean  MODIS Data
Above we walked through the specific workflow for using MODIS data.
You can however use the same function that you used last week 
to process Landsat and NAIP data for MODIS.

The difference is now you  an  add the v ariable  parameter
to include the list of bands that you wish to grab from the
data.

In [ ]:
# Function in your homework assignment
def open_clean_bands(band_path,
                     crop_bound,
                     valid_range=None,
                     variable=None):
    """Open and clean a single landsat band.

    Parameters
    -----------
    band_path : string 
        A path to the array to be opened.
    crop_bound : geopandas GeoDataFrame
        A geopandas dataframe to be used to crop the raster data using rioxarray clip().
    valid_range:tuple (optional)
        A tuple of min and max range of values for the data. Default = None.
    variable : List
        A list of variables to be opened from the raster.

    Returns
    -----------
    band : xarray DataArray
        Cropped xarray DataArray

    """

    crop_bound_box = [box(*crop_bound.bounds.loc[0])]

    # I've added another check to this function
    import rasterio as rio

    with rio.open(band_path) as src:
        driver = src.driver

    if driver == "HDF4" and variable is None:
        raise ValueError("OOPS I've encountered an HDF file  "
                         "which may have subgroups, please be sure  "
                         "to specify the variable=parameter so "
                         "I can clip the data properly.")

    try:
        band = rxr.open_rasterio(band_path,
                                 masked=True,
                                 variable=variable).rio.clip(crop_bound_box,
                                                             crs=crop_bound.crs,
                                                             all_touched=True,
                                                             from_disk=True).squeeze()
    except:
        raise ValueError(
            "Oops - I couldn't clip your data. This may be due to a crs error.")

    # Only mask the data to the valid range if a valid range tuple is provided
    if valid_range is not None:
        mask = ((band < valid_range[0]) | (band > valid_range[1]))
        band = band.where(~xr.where(mask, True, False))

    return band

Above i've modified the function that we gave you to  
check to see if the data are in HDF4 format  - it  
returns an error if you don't specify the  variable  objects to
extract. 

In [ ]:
# # If you dont specify variables, this function will fail to clip the data
# Uncomment this  cell to run the code (which should fail)
# open_clean_bands(modis_pre_path,
#                  crop_bound=fire_boundary)

Below, we use the bands object to subset the data. 

In [ ]:
# Remember this object created above
rgb_bands

In [ ]:
cleaned_MODIS_data = open_clean_bands(modis_pre_path,
                                      crop_bound=fire_boundary,
                                      variable=desired_bands)

Note - it's ok if the plot below looks funky.
It represents very few pixels and pre-fire data does
not have a lot of variation in the pixels.

In [ ]:
# Plot unclipped data - RGB (just a demo)
#  This  plot may look funky
modis_rgb = cleaned_MODIS_data[rgb_bands].to_array().squeeze()
modis_rgb_plot = clean_array_plot(modis_rgb)

# Plot MODIS RGB image -Note that this looks weird only because
# the data are clipped to such a "small" extent
ep.plot_rgb(modis_rgb_plot,
            rgb=[0, 1, 2],
            title='RGB Image of MODIS Data',
            figsize=(7, 7))

plt.show()

In [ ]:
ep.plot_bands(modis_rgb_plot)
plt.show()